# Pràctica 2: Generant dades d'entrenament

## Introducció

En aquest notebook generarem dades per entrenar una xarxa neuronal per controlar el robot per recorrer el circuit i evitar els obstacles. Farem servir les dades que generem en aquest notebook per entrenar una xarxa neuronal en el següent notebook.

In [3]:
!pip install aitk.robots aitk.networks tensorflow numpy

In [4]:
import aitk.robots as bots

In [5]:
world = bots.World(240, 150)
# world.add_wall("blue", 60, 40, 65, 110)
# world.add_wall("blue", 65, 70, 200, 75)
# #world.add_wall("blue", 100, 0, 140, 30, box=False)
# world.add_wall("blue", 100, 0, 105, 30)

# world.add_wall("blue", 140, 110, 145, 150)
# world.add_wall("blue", 195, 50, 200, 70)

world.add_wall("blue", 0, 0, 50, 50)
world.add_wall("blue", 75, 200, 125, 150)
world.add_wall("blue", 150, 0, 200, 50)
world.add_wall("blue", 150, 150, 200, 100)
world.add_wall("blue", 0, 100, 50, 150)

world.add_wall("blue", 100, 25, 105, 150)
world.add_wall("blue", 100, 70, 120, 75)

world.add_bulb("yellow", 125, 130, 1, 30)

robot = bots.Scribbler(x=30, y=80, a=90)
robot.add_device(bots.RangeSensor(width=45,max=20,name="front"))
robot.add_device(bots.RangeSensor(width=45,max=20,position=(6,-6),
                                         a=90,name="left"))
robot.add_device(bots.LightSensor(position=(6, 0), name="light"))

world.add_robot(robot)

world.watch()

Random seed set to: 8646705


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

## Creant ubicacions d'inici aleatòries úniques

En aquesta pràctica, volem generar un conjunt de dades d'entrenament per a la nostra xarxa neuronal. Per generar un conjunt de dades d'entrenament robust, ens agradaria que el robot comencés des de moltes ubicacions diferents al món. Podeu utilitzar `robot.set_random_pose()` per establir la posició del robot en una ubicació aleatòria al món. No obstant això, aquesta funció a vegades establirà la posició del robot en la mateixa ubicació que abans. Per tant, necessitem crear una funció que establirà la posició del robot en una ubicació aleatòria, però no la posarà en la mateixa ubicació que abans.

In [6]:
def inici_aleatori(world):
    """Estableix la posició del robot a una ubicació aleatòria al món."""
    world.robots[0].set_random_pose() # Pose aleatòria.
    world.update() # Actualitza el món per mostrar la nova posició.

### Testeja la funció `inici_aleatori`

Per començar, implementeu la funció `inici_aleatori` que establirà la posició del robot en una ubicació aleatòria, però no la posarà en la mateixa ubicació que abans. Aquesta funció hauria de ser cridada per a cada episodi d'entrenament. Per exemple, si volem generar 100 episodis d'entrenament, cridarem aquesta funció 100 vegades per generar 100 ubicacions d'inici úniques.

In [7]:
from time import sleep

for i in range(10):
    inici_aleatori(world)
    sleep(1)

### Generant valors de moviment i rotació

En la pràctica anterior, vau controlar directament el robot cridant `robot.move(translate, rotate)`. Ara volem moure el robot i guardar els moviments en un fitxer. Per fer-ho, crearem una funció anomenada `determine_move` (basada en l'algorisme `wall_follow`) que prengui els valors actuals dels sensors i retorni una tupla que representi els quantitats de translació i rotació apropiades per a aquesta situació.

Has de parar quan la volta estigui completada. Per aturar-te, pots utilitzar la funció `robot.move(0, 0)`. Pots utilitzar la funció `robot.get_pose()` per obtenir la posició actual del robot (x, y, a). Pots utilitzar la funció `distance(x1, y1, x2=None, y2=None)` (del paquet `utils`) per obtenir la distància des de la ubicació actual del robot fins a una ubicació donada. Pots utilitzar la funció `robot.set_pose()` per establir la posició del robot (x, y, a) a un valor nou.


In [8]:
def determine_move(robot):
    """
    Determina el moviment del robot en funció de les seves lectures de sensors.
    Retorna una tupla amb la velocitat de translació, la velocitat de rotació i un booleà que indica si s'ha de parar.

    El robot ha de seguir una paret fins que torni a la posició inicial.
    """


#### Testejant la funció `determine_move`

Hem de provar la funció `determine_move` i verificar que encara controla adequadament el robot per seguir la paret. Per fer-ho, cridarem la funció `determine_move` en un bucle i cridarem `robot.move` amb els valors de moviment retornats per la funció `determine_move`. Això hauria de fer que el robot segueixi la paret fins que la volta estigui completada.

In [9]:
def exit_maze(robot):    
    translate, rotate, stop, dist_esq, dist_davant, llum = determine_move(robot)
    robot.move(translate, rotate)

    if stop:
        print("STOP")
        return True


In [10]:
inici_aleatori(world)

world.reset()

world.seconds(180, [exit_maze], real_time=False)

Using random seed: 8646705


  0%|          | 0/1800 [00:00<?, ?it/s]

STOP
Simulation stopped at: 00:01:25.10; speed 20.79 x real time


True

### Generant dades d'entrenament 

Ara tenim totes les peces en el lloc per generar les dades d'entrenament.

Volem generar un conjunt de dades d'entrenament per a la nostra xarxa neuronal i el simulador en temps real serà massa lent. Per tant, utilitzarem la versió no en temps real del simulador. Per aixó utilitzarem `realtime = False` quan cridem a `world.seconds`.

També hem de tindre en que, per entrenar una xarxa neuronal, tots els valors de les dades haurien d'estar normalitzats per ajustar-se a l'interval de la funció d'activació que utilitzarem. Farem servir una funció d'activació amb un interval de [-1,1], ja que les comandes del motor poden ser negatives o positives. Els valors del sensor de sònar estan en l'interval [0,20]. Haureu de trobar els valors mínims i màxims per normalitzar les dades abans de desar-les al fitxer.

En cada línia del fitxer que crearem hi haurà valors dels sensors, separats per espais en blanc, seguits de comandes del motor separades per espais en blanc, acabant amb un salt de línia.

Un problema que potencialment hem d'abordar és que el nostre programa de recòrrer el laberint pot quedar-se aturat en certes situacions o anar enrere. Hem de construir la nostra funció de generació de dades amb això en ment. Per exemple, si el robot es queda aturat, no volem que les dades d'entrenament reflecteixin aquesta situació.

Crearem la funció `generate_data` que generi una llista de dades d'entrenament. Aquesta funció hauria de generar un nombre fix de dades d'entrenament. Per cada iteració d'entrenament, la funció hauria de cridar `inici_aleatori` per establir la posició del robot en una ubicació aleatòria, reiniciar el mon i executar una simulació.

In [11]:
        

def generate_data(robot, world, trials):
    """Aquí haurà de generar les dades per entrenar la xarxa neuronal."""
        

In [12]:
generate_data(robot, world, 20)

Using random seed: 8646705


  0%|          | 0/2400 [00:00<?, ?it/s]

STOP
Simulation stopped at: 00:00:46.90; speed 13.3 x real time
Using random seed: 8646705


  0%|          | 0/2400 [00:00<?, ?it/s]

STOP
Simulation stopped at: 00:00:46.90; speed 12.34 x real time
Using random seed: 8646705


...

STOP
Simulation stopped at: 00:00:46.90; speed 16.67 x real time
Using random seed: 8646705


  0%|          | 0/2400 [00:00<?, ?it/s]

STOP
Simulation stopped at: 00:00:46.90; speed 11.94 x real time
Using random seed: 8646705


  0%|          | 0/2400 [00:00<?, ?it/s]

STOP
Simulation stopped at: 00:00:46.90; speed 33.51 x real time


In [13]:
data

[[10.57638297494919, 20.0, 0.0690139683385698, 0.5, 0],
 [10.708140830913536, 20.0, 0.06936323750787081, 0.5, 0],
 [10.971656542842158, 20.0, 0.07005774593047209, 0.5, 0],
 [11.366930110735149, 20.0, 0.07108886853569633, 0.5, 0],
 [11.893961534592451, 20.0, 0.07244225547999261, 0.5, 0],
 [12.552750814414065, 20.0, 0.07409614628388383, 0.5, 0],
 [13.211540094235664, 20.0, 0.07570363429643548, 0.5, 0],
 [13.870329374057292, 20.0, 0.07726010388758413, 0.5, 0],
 [14.529118653878921, 20.0, 0.07876101367919192, 0.5, 0],
 [15.187907933700535, 20.0, 0.080201918444728, 0.2, 0.4],
 [15.324375999364822, 20.0, 0.08017341564264387, 0.2, 0.4],
 [14.980556967036632, 20.0, 0.07866049317335519, 0.5, 0],
 [15.098457970136215, 20.0, 0.07811839002214024, 0.2, 0.4],
 [14.798637482486704, 20.0, 0.07624697718509688, 0.5, 0],
 [14.897472462413392, 20.0, 0.07522888364212336, 0.5, 0],
 [15.365845811656943, 20.0, 0.07509756058291811, 0.2, 0.4],
 [15.456820458853796, 20.0, 0.07384761826982465, 0.2, 0.4],
 [15.209

Processarem les dades d'entrenament en la següent cel·la. Aplicarem les següents transformacions:

- Normalitzarem les dades del sensor i les comandes del motor per ajustar-se a l'interval de la funció d'activació. En el nostre cas, utilitzarem la funció d'activació tanh, que té un interval de [-1,1] i solament cal normalitzar les dades dels sensors de proximitat.
- Llevarem les dades on el robot estigui aturat o vagi enrere. 

In [14]:
processed_lines = []

for line in data:
    # Processem les dades per tal de tenir un format més adequat per a la xarxa neuronal.
    

with open("training_data.txt", "w") as f:
    f.writelines(processed_lines)
